In [1]:
from OpenSSL import crypto
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import os

In [2]:
def get_existing_certs(distinct_urls,folder):
    successes = []
    data_path = folder+'/'
    for url in distinct_urls:
        try:
            f = open(data_path+url)    
            if f.readline() != '':
                successes.append(url)
        except:
            pass
    return successes

In [1]:
path = "benign_unpopular20000.csv"
df = pd.read_csv(path)
urls = df['Domain']

NameError: name 'pd' is not defined

In [4]:
how_many = 50000
urls = urls[:50000]

In [5]:
data_path = 'benign_data'
successes = get_existing_certs(urls,data_path)
len(successes)

41680

In [6]:
missing_cert_urls = []
found_url_indices = np.zeros(len(successes),dtype=bool)
certs = []
creation_times = []
for i,url in enumerate(successes):
    cert_file = 'benign_data/'+url
    
    f = open(cert_file)
    line = f.readline()
    found_cert = False
    while line != '':
        line = f.readline()
        if "-----BEGIN CERTIFICATE-----" in line:
            found_cert=True
            line = ''
    f.close()
    if found_cert:
        found_url_indices[i] = True
        cert = crypto.load_certificate(crypto.FILETYPE_PEM, open(cert_file).read())
        certs.append(cert)
        t = os.path.getctime(cert_file)
        creation_times.append(t)
    else:
        missing_cert_urls.append(cert)
len(missing_cert_urls)

NameError: name 'f' is not defined

In [ ]:
final_df = pd.DataFrame()

subjects = [cert.get_subject() for cert in certs]
issuers = [cert.get_issuer() for cert in certs]

expiries = [cert.has_expired() for cert in certs]

In [ ]:
Counter(expiries)

In [ ]:
final_df['CN'] = [s.CN for s in subjects]
final_df['O'] = [s.O for s in subjects]
final_df['C'] = [s.C for s in subjects]
final_df['businessCategory'] = [s.businessCategory for s in subjects]
final_df['serialNumber'] = [s.serialNumber for s in subjects]
final_df['juristiction'] = [s.jurisdictionLocalityName for s in subjects]
final_df['issuer_CN'] = [i.CN for i in issuers]
final_df['issuer_O'] = [i.O for i in issuers]
final_df['expired'] = expiries
final_df['notBefore'] = [c.get_notBefore() for c in certs]
final_df['notAfter'] = [c.get_notAfter() for c in certs]

verifications = []
for i,url in enumerate(successes):
    if found_url_indices[i]:
        f = open(data_path+'/'+url)
        line = f.readline()
        verified=False
        while line != '':
            if "Verify return code:" in line:
                if verified:
                    #print(url)
                    pass
                else:
                    verifications.append(line)
                    verified=True
            line = f.readline()
        if not verified:
            #print("FAILED: "+url)
            verifications.append('Verify return code: None (none)')
        f.close()  
new_verifications = [v[5:].split(' ')[3] for v in verifications]
final_df['verCode'] = new_verifications
final_df['time_collected'] = creation_times

In [ ]:
final_df

In [ ]:
final_df.to_csv('final_certificate_datasets/benign_certs.csv')